## Data preprocessing

source: https://archive.ics.uci.edu/dataset/186/wine+quality

### Imports

In [78]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

### Data load

In [79]:
df_red = pd.read_csv("data/winequality-red.csv", sep=";")
df_white = pd.read_csv("data/winequality-white.csv", sep=";")

### Data catenation
Those data have one cathegorical variable _type_. As there are two .csv files, one for each variable, it is neccessary to transform two files without _type_ variable to one table with new column(s) to capture the new variable _type_. Because there is just one cathegorical variable which has only two values, the transformation could be done using on-hot trick without significant size/dimension affection of the new data.

In [80]:
# add new on-hot columns to each dataset
df_red["red"] = 1
df_red["white"] = 0

df_white["red"] = 0
df_white["white"] = 1

# catenate both datasets together
df = pd.concat([df_red, df_white], ignore_index=True)

# catenation were successful, column names and row count as expected
# print(df.info())
# print(df.head())

### Data preprocessing
There are a lot of data preprocessing methods which should be used in ussual case. But in this dataset, a lot of potentional problems cannot occur. There is not any _date_ variable, neither _strings_ or other objects. Using the _info()_ method of pandas dataframe, we can see that the dataset has none of _None_ or _NAN_ values, so all are presented, and there are also just _int_ or _float_ variables. Moreover, all columns have same number of rows.

In [81]:
# there is not any None value in any column
print(df.isna().sum())

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
red                     0
white                   0
dtype: int64


### Remove duplicates

In [82]:
# remove duplicates
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(6497, 14)
(5320, 14)


### Unite units
Those two columns contains values in mg/dm3 instead of g/dm3. Thus, those are converted.

In [83]:
df["free sulfur dioxide"] = df["free sulfur dioxide"] / 1000
df["total sulfur dioxide"] = df["total sulfur dioxide"] / 1000

### Standardize the features
For the predicted value _quality_ and the integer/bool values of _red_ and _white_ it does not make any sense to do the standardization.

In [84]:
# Standardize features
features = df.drop(columns=["quality", "red", "white"]).columns
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

### Normalize the features
Normalisation of all columns, except the quality, where we need the original values. The normalisation of _red_ and _white_ column does nothing.

In [85]:
# Scale the features and target variable to the range [0, 1]
# print(df.describe())
features = df.drop(columns=["quality"]).columns
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])
# print(df.describe())

### Outliers detection
We tried z-score and IQR to detect and get rid of outliers. Since the IQR perform better, we chose it for the final use.

In [86]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Define outliers
outliers = ((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)

# Remove outliers
df = df[~outliers]

In [87]:
# put wine quality as a final column
cols = list(df.columns)
cols[-3], cols[-1] = cols[-1], cols[-3]
df = df[cols]

### Save prepared data

In [88]:
df.to_csv("data/wine_prepared.csv", index=False)    # ignore indices while saving